### Part III Overview:
Goal: Provide a basic info of the city and best recommendations of restaurants, hotels and landmarks to the users

### Steps:
* create functions to extract city information from the wikipedia page
* ask for user inputs
* use the user inputs to query the database

In [1]:
import json, ast
import pandas as pd
from django.utils.encoding import smart_str, smart_unicode
import requests_cache
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import display
from ipywidgets import widgets
import sqlite3 as sql

### Extract Info From Wikipedia Page

In [2]:
requests_cache.install_cache("cache")

def modify(city):
    """
    modify the input
    
    Argument: city
    
    Return: soup from the url link
    """
    # capitalize the first letter of each word in a string
    city = city.title()
    # replace the empty space with the _
    city = city.strip().replace(' ', '_')
    url = "https://en.wikipedia.org/wiki/" + str(city) +",_California"
    info = requests.get(url).content
    wikisoup = BeautifulSoup(info, 'lxml')
    return wikisoup

In [3]:
def extract_overview(city):
    """
    extract a brief overview from the wikipedia page
    
    Argument: city
    
    Return: print the overview description
    """
    wikisoup = modify(city)
    content = wikisoup.findAll("p")
    if content[0].text == "California":
        brief = content[1].text
    else:
        brief = content[0].text
    print brief

In [4]:
def extract_info(city):
    """
    extract the basic information of the city
    
    Argument: city
    
    Return: a dataframe with the basic information
    """
    wikisoup = modify(city)
    data1 = wikisoup.findAll("tr", {"class": "mergedrow"})
    data = [dat.text for dat in data1]
    split = [dat.strip("\n") for dat in data]
    pd_info = pd.DataFrame(split)
    pd_info.rename(columns = {list(pd_info)[0]: 'info'}, inplace = True)

    #remove []
    pd_info['info'] = pd_info['info'].map(lambda x: re.sub("(\[).*?([\]])", "", x))
    #remove unwanted characters
    pd_info['info'] = pd_info['info'].map(lambda x: re.sub(ur'[\•]', ' ', x))
    pd_info['info'] = pd_info['info'].str.strip("\n")
    pd_info['info'] = pd_info['info'].str.strip()
    pd_info['info'] = pd_info['info'].str.replace("\n", " : ")
    pd_info['info'] = pd_info['info'].str.replace(" [: ]+", " : ")
    return pd_info

### Query Database

In [5]:
def recommend_res(city, res_cat, res_rating):
    city = city.title()
    visit_city = city + ", CA"
    res_cat = res_cat.title()
    rat_1 = res_rating[0]
    rat_2 = res_rating[-1]
    print "You enter City: %s, Restaurant Category: %s, Lower_bound Rating: %s, Upper_bound Rating: %s\n" %(visit_city, res_cat, rat_1, rat_2)
    conn = sql.connect("accommodation.sqlite")
    res_sql = '''SELECT categories, city, name, rating, review_count, snippet_text,url FROM Best_Restaurants WHERE city = '%s' 
                    AND categories = '%s' AND rating BETWEEN '%s' AND '%s' ''' %(visit_city, res_cat, rat_1, rat_2)
    restaurants = pd.read_sql(res_sql, conn)
    
    res_sql_alternative = '''SELECT categories, city, name, rating, review_count, snippet_text,url FROM Best_Restaurants WHERE city = '%s' AND rating BETWEEN '%s' AND '%s' LIMIT 5''' %(visit_city, rat_1, rat_2)
    restaurants_alternative = pd.read_sql(res_sql_alternative, conn)
    
    if restaurants.empty:
        print("We are sorry. We can not find a restaurant that meets your criterias in our database. Please try a different kinds of food or rating. But first, let's look at the TOP 5 alternatives that we find.")
        return restaurants_alternative
    else:
        return restaurants
    conn.close()

In [6]:
def recommend_hotel_land(city, cat, rating, accommodation):
    city = city.title()
    visit_city = city + ", CA"
    cat = cat.title()
    if cat[-1] == 's':
        cat = cat[:-1]
    rat_1 = rating[0]
    rat_2 = rating[-1]
    print "You enter City: %s, Category: %s, Lower_bound rating: %s, Upper_bound rating: %s\n" %(visit_city, cat, rat_1, rat_2)
    conn = sql.connect("accommodation.sqlite")
    if 'hotels' in accommodation.lower():
        hotel_land_sql = '''SELECT categories, city, name, rating, review_count, snippet_text,url FROM Best_''' + accommodation + ''' WHERE city = '%s' 
                    AND name LIKE '%%%s%%' AND rating BETWEEN '%s' AND '%s' ''' %(visit_city, cat, rat_1, rat_2)
    
    else:
        hotel_land_sql = '''SELECT categories, city, name, rating, review_count, snippet_text,url FROM Best_''' + accommodation + ''' WHERE city = '%s' 
                    AND categories LIKE '%%%s%%' AND rating BETWEEN '%s' AND '%s' ''' %(visit_city, cat, rat_1, rat_2)
        
    hotel_land = pd.read_sql(hotel_land_sql, conn)
    
    hotel_land_sql_alternative = '''SELECT categories, city, name, rating, review_count, snippet_text,url FROM Best_''' + accommodation + ''' WHERE city = '%s' 
                    AND categories LIKE '%%%s%%' AND rating BETWEEN '%s' AND '%s' ''' %(visit_city, accommodation, rat_1, rat_2)
    
    hotel_land_alternative = pd.read_sql(hotel_land_sql_alternative, conn)
    
    if hotel_land.empty:
        print("We are sorry. We can not find a result that meets your criterias in our database. Please try a different input or rating. But first, let's look at the TOP 5 alternatives that we find.")
        return hotel_land_alternative
    else:
        return hotel_land
    conn.close()

### Ask User Inputs

In [7]:
city = raw_input("Please enter the city that you want to visit: ")
print "\nWelcome to the city of", city.title() + "! We are excited to provide you an ideal travel guide to help you explore this wonderful city!\n"
print "Here is some basic info of the city of", city.title() + "!\n"
extract_overview(city)
extract_info(city)

Please enter the city that you want to visit: san francisco

Welcome to the city of San Francisco! We are excited to provide you an ideal travel guide to help you explore this wonderful city!

Here is some basic info of the city of San Francisco!

San Francisco (SF) (/sæn frənˈsɪskoʊ/, Spanish for Saint Francis; Spanish: [san fran.ˈθis.ko]), officially the City and County of San Francisco, is the cultural, commercial, and financial center of Northern California. It is the birthplace of the United Nations.[23][24][25] Located at the north end of the San Francisco Peninsula, San Francisco is about 47.9 square miles (124 km2)[17] in area, making it the smallest county—and the only consolidated city-county[26]—within the state of California. With a density of about 18,451 people per square mile (7,124 people per km2), San Francisco is the most densely settled large city (population greater than 200,000) in California and the second-most densely populated major city in the United States aft

,info
0,"Motto: Oro en Paz, Fierro en Guerra (Spanish) ..."
1,State : California
2,CSA : San Jose–San Francisco–Oakland
3,Metro : San Francisco–Oakland–Hayward
4,"Incorporated : April 15, 1850"
5,Founded by : José Joaquin Moraga : Francisco P...
6,Named for : St. Francis of Assisi
7,Type : Mayor-council
8,Body : Board of Supervisors
9,Mayor : Edwin M. Lee (D)


In [8]:
def ask_input():
    """
    get the user input
    
    Argument: None
    
    Return: recommended content in dataframe format
    """
    print "\nPlease answer the following questions so that we can recommend the best restaurants, hotels and landmarks to you.\n"
    
    accommodation = raw_input("Which accommondation do you want us to recommend? For example, restaurants, hotels or landmarks\n")
    
    if accommodation.title() in "Restaurants":
        res_cat = raw_input("What is your favorite kinds of food? \n")
        res_rating = raw_input("What is your preferred range of rating for restaurants? \n")
        output = recommend_res(city, res_cat, res_rating)
    elif accommodation.title() in "Hotels":
        hotel_cat = raw_input("What kind of hotel do you prefer to stay in? For example, inns, hotels or resorts? \n")
        hotel_rating = raw_input("What is your preferred range of rating for hotels? \n")
        output = recommend_hotel_land(city, hotel_cat, hotel_rating, 'Hotels')
    else:
        land_cat = raw_input("What kind of landmarks are you looking for? \n")
        land_rating = raw_input("What is your preferred range of rating for landmarks? \n")
        output = recommend_hotel_land(city, land_cat, land_rating, 'Landmarks')
    return output

In [9]:
def verify_satisfaction():
    """
    verify whether the users are satisfied with the results
    
    Argument: None
    
    Return: ask_input() function
    """
    satisfied = raw_input("Are you satisfied with the results? \n")

    if satisfied.title() in "Yes":
        print ("Thanks for using our system. We wish you have a wonderful trip.\n")
    else:
        print("Ok, let's try again.\n")
        return ask_input()

In [10]:
ask_input()


Please answer the following questions so that we can recommend the best restaurants, hotels and landmarks to you.

Which accommondation do you want us to recommend? For example, restaurants, hotels or landmarks
restaurants
What is your favorite kinds of food? 
chinese
What is your preferred range of rating for restaurants? 
1,4
You enter City: San Francisco, CA, Restaurant Category: Chinese, Lower_bound Rating: 1, Upper_bound Rating: 4

We are sorry. We can not find a restaurant that meets your criterias in our database. Please try a different kinds of food or rating. But first, let's look at the TOP 5 alternatives that we find.


,categories,city,name,rating,review_count,snippet_text,url
0,Ramen,"San Francisco, CA",Hinodeya Ramen Bar,4.0,182,"Yum. Another ramen bar! With, of course, a wai...",https://www.yelp.com/biz/hinodeya-ramen-bar-sa...
1,Ramen,"San Francisco, CA",Nojo Ramen Tavern,4.0,230,There was no wait for lunch on a weekend. The ...,https://www.yelp.com/biz/nojo-ramen-tavern-san...
2,Japanese,"San Francisco, CA",Kui Shin Bo,4.0,478,At this point the wife and I have been out of ...,https://www.yelp.com/biz/kui-shin-bo-san-franc...
3,Cocktail Bars,"San Francisco, CA",Horsefeather,4.0,82,Solid place for reasonably priced fancy cockta...,https://www.yelp.com/biz/horsefeather-san-fran...
4,Seafood,"San Francisco, CA",Fog Harbor Fish House,4.0,2920,First I have to tell you that I usually don't ...,https://www.yelp.com/biz/fog-harbor-fish-house...


In [11]:
verify_satisfaction()

Are you satisfied with the results? 
no
Ok, let's try again.


Please answer the following questions so that we can recommend the best restaurants, hotels and landmarks to you.

Which accommondation do you want us to recommend? For example, restaurants, hotels or landmarks
hotels
What kind of hotel do you prefer to stay in? For example, inns, hotels or resorts? 
resorts
What is your preferred range of rating for hotels? 
1, 4
You enter City: San Francisco, CA, Category: Resort, Lower_bound rating: 1, Upper_bound rating: 4

We are sorry. We can not find a result that meets your criterias in our database. Please try a different input or rating. But first, let's look at the TOP 5 alternatives that we find.


,categories,city,name,rating,review_count,snippet_text,url
0,Hotels,"San Francisco, CA",The Scarlet Huntington,4.0,245,This is the best hotel that I have stayed at i...,https://www.yelp.com/biz/the-scarlet-huntingto...
1,Hotels,"San Francisco, CA",Hotel Abri,4.0,273,"Love!\n\nEasy to book online, we arrived and t...",https://www.yelp.com/biz/hotel-abri-san-franci...
2,Hotels,"San Francisco, CA",JW Marriott San Francisco Union Square,4.0,302,A friend from out of town was staying at the J...,https://www.yelp.com/biz/jw-marriott-san-franc...
3,Hotels,"San Francisco, CA",Union Hotel,4.0,25,"Ok. Here it goes: I visited San Francisco, and...",https://www.yelp.com/biz/union-hotel-san-franc...
4,Hotels,"San Francisco, CA",Hotel Del Sol,4.0,120,Pros:\n-super family friendly\n-kids loved the...,https://www.yelp.com/biz/hotel-del-sol-san-fra...
5,Hotels,"San Francisco, CA","The Ritz-Carlton, San Francisco",4.0,375,This review is a longtime coming - we had our ...,https://www.yelp.com/biz/the-ritz-carlton-san-...
